In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
month = '202011'

In [3]:
from skinet.utils.athena_tools import *
conn = athena_connection()
df = load_from_athena(conn, '''

    SELECT DISTINCT a.kcalss
     , a.conn_dept
     , a.conn_emp
     , a.cost_dept
     , a.cost_emp
     , a.ctnum
     , a.cstat
     , a.gad_name
     , a.gad_start
     , a.conn_start
     , a.conn_end
     , CAST(a.monexp_amt AS DOUBLE) AS monexp_amt
     , a.conn_type
     , a.ctype_l
     , a.ctype_m
     , a.intro_asp
     , a.itype_l
     , a.itype_m
     , a.itype_s
     , a.itype_sdesc
     , a.kunnr
     , c.name1
     , a.gad_add01
     , a.post_add01
     , a.first_bildate
     , c.telf2
--     , (SELECT ETAX_EMAIL FROM SAP.ZSDT4021 WHERE CTNUM = A.CTNUM AND RQNUM = A.RQNUM AND AMT_TYPE = 'F3' AND MANDT = '100') AS 전자계산서EMAIL
--     , (SELECT GROS_CODE FROM SAP.ZSDT1110 WHERE KUNNR = A.KUNNR AND MANDT = '100' AND BUKRS = '1000') AS GROSCODE
     , a.svc_str
     , a.pay_type
     , a.depo_type
     , a.line_type
     , a.pck_str
--     , (SELECT BIR_DATE FROM SAP.ZSDT1110 WHERE KUNNR = A.KUNNR AND MANDT = '100' AND BUKRS = '1000') AS 생년월일
     , SUBSTR(c.stcd1, 1 ,6) AS B_YMD
     , SUBSTR(c.stcd1, 7 ,1) AS SEX
     , c.stcd2
     , a.gros_code
     , a.gross_type
     , a.kvgr4
     , a.gad_pstlz
     , a.gad_add02
     , a.post_add02
     , b.result1
     , b.result2
     , b.result3
     , b.result4
     , b.result5
     , b.result6
     , b.sms_yn
     , b.email_yn
     , b.tm_yn
     , b.dm_yn
     , b.sk1_yn
     , b.sk2_yn
     , b.adt1_yn
     , b.aedat
     , b.aenam
  FROM sap.zsdt4013      AS a 
  LEFT JOIN (SELECT ctnum, kunnr, sms_yn, email_yn, tm_yn, dm_yn, sk1_yn, sk2_yn, adt1_yn, aedat, aenam, result1, result2, result3, result4, result5, result6 FROM sap.zsdt3748 WHERE usest = 'X') AS b
        ON a.ctnum = b.ctnum
  LEFT JOIN (SELECT kunnr, stcd2, stcd1, telf2, name1 FROM sap.kna1 WHERE mandt = '100') AS c
        ON a.kunnr = c.kunnr
  WHERE a.mandt = '100' 
    AND a.stmon = \'{}\'  --년월 변경
    AND a.bukrs = '1000'
    AND a.cstat IN ('1','2');

'''.format(month)) 

...tmp_db name = tmp_20201207_10_17_22_107eb78ec09969947151f8dc510d23e8fae9a2b9
Start creating table
drop table if exists default.tmp_20201207_10_17_22_107eb78ec09969947151f8dc510d23e8fae9a2b9
<< table not exist

        create  table         default.tmp_20201207_10_17_22_107eb78ec09969947151f8dc510d23e8fae9a2b9
        with (format='Parquet', parquet_compression='GZIP')
        as
        

    SELECT DISTINCT a.kcalss
     , a.conn_dept
     , a.conn_emp
     , a.cost_dept
     , a.cost_emp
     , a.ctnum
     , a.cstat
     , a.gad_name
     , a.gad_start
     , a.conn_start
     , a.conn_end
     , CAST(a.monexp_amt AS DOUBLE) AS monexp_amt
     , a.conn_type
     , a.ctype_l
     , a.ctype_m
     , a.intro_asp
     , a.itype_l
     , a.itype_m
     , a.itype_s
     , a.itype_sdesc
     , a.kunnr
     , c.name1
     , a.gad_add01
     , a.post_add01
     , a.first_bildate
     , c.telf2
--     , (SELECT ETAX_EMAIL FROM SAP.ZSDT4021 WHERE CTNUM = A.CTNUM AND RQNUM = A.RQNUM AND AMT_

In [4]:
hashed = load_from_athena(conn, '''
SELECT * 
  FROM adt_work.hash_info
''') 

...tmp_db name = tmp_20201207_10_19_15_f91c5b6e1ffcd079a8de18ec144852ec1261ffe0
Start creating table
drop table if exists default.tmp_20201207_10_19_15_f91c5b6e1ffcd079a8de18ec144852ec1261ffe0
<< table not exist

        create  table         default.tmp_20201207_10_19_15_f91c5b6e1ffcd079a8de18ec144852ec1261ffe0
        with (format='Parquet', parquet_compression='GZIP')
        as
        
SELECT * 
  FROM adt_work.hash_info

..
>> Athena Query Success!!
>> Athena Query EXCUTION COMPLETE....
>> S3 FILE GENERATION SUCCESS!! (File count: 9)
Elapsed Time = 19.62
>> Start downloading table
aws s3 cp s3://adt-sandbox/dsc/tables/cf784f4d-c7fe-46e6-b1f9-d3966daf4900 /home/skinet/tmp/tmp_20201207_10_19_15_f91c5b6e1ffcd079a8de18ec144852ec1261ffe0 --recursive
Elapsed Time = 12.14
>> Loading data
Dropping tmp db
<< Delete s3://adt-sandbox/dsc/tables/cf784f4d-c7fe-46e6-b1f9-d3966daf4900
Removing tmp_dir!
Elapsed Time for downloading and loading = 9.29
Total Elapsed Time = 41.05


In [5]:
full = pd.merge(df, hashed, how='left', left_on='ctnum', right_on='ctnum_hash')

In [6]:
full = full[['kunnr_y', 'ctnum_y', 'conn_dept', 'conn_emp', 'cost_dept', 'cost_emp', 
             'cstat', 'gad_name', 'gad_start', 'conn_start', 'conn_end', 'monexp_amt',
             'conn_type', 'ctype_l', 'ctype_m', 'intro_asp', 'itype_l', 'itype_m', 
             'itype_s', 'itype_sdesc', 'name1', 'gad_add01', 'post_add01', 'first_bildate',
             'telf2', 'svc_str', 'pay_type', 'depo_type', 'line_type', 'pck_str', 'b_ymd',
             'sex', 'stcd2', 'gros_code', 'gross_type', 'kvgr4', 'gad_pstlz', 'gad_add02', 
             'post_add02', 'result1', 'result2', 'result3', 'result4', 'result5', 'result6',
             'sms_yn', 'email_yn', 'tm_yn', 'dm_yn', 'sk1_yn', 'sk2_yn']]

In [7]:
# cond1 = (full['conn_type'] == 'A01') | (full['conn_type'] == 'A02')
# cond2 = (full['gross_type'] != "Y")                
# cond3 = (full['gros_code'] == ' ')         

In [8]:
# full = full[cond1 & cond2 & cond3]

In [9]:
# code_table = pd.read_csv('/home/skinet/hojin/코드테이블.csv')

In [10]:
# channel_l = code_table[code_table['table'] == '채널(대)'][['code', 'value']]

In [11]:
# full = pd.merge(full, channel_l, how='left', left_on='ctype_l', right_on='code')
# full.rename(columns={'value' : '채널(대)'}, inplace=True)
# full.drop(['code', 'ctype_l'], axis=1, inplace=True)

In [12]:
# upjong_m = code_table[code_table['table'] == '업종(중)'][['code', 'value']]

In [13]:
# full = pd.merge(full, upjong_m, how='left', left_on='itype_m', right_on='code')
# full.rename(columns={'value' : '업종(중)'}, inplace=True)
# full.drop(['code', 'itype_m'], axis=1, inplace=True)

In [14]:
# napip = code_table[code_table['table'] == '납입방법'][['code', 'value']] 

In [15]:
# full = pd.merge(full, napip, how='left', left_on='pay_type', right_on='code')
# full.rename(columns={'value' : '납입방법'}, inplace=True)
# full.drop(['code', 'pay_type'], axis=1, inplace=True)

In [16]:
# ipgum = code_table[code_table['table'] == '입금방법'][['code', 'value']] 

In [17]:
# ipgum.code = ipgum.code.str.zfill(2)

In [18]:
# full = pd.merge(full, ipgum, how='left', left_on='depo_type', right_on='code')
# full.rename(columns={'value' : '입금방법'}, inplace=True)
# # full.drop(['code', 'depo_type'], axis=1, inplace=True)

In [19]:
# upjong_s = code_table[code_table['table'] == '업종(소)'][['code', 'value']] 

In [20]:
# upjong_s.code = upjong_s.code.str.zfill(3)

In [21]:
# full = pd.merge(full, upjong_s, how='left', left_on='itype_s', right_on='code')
# full.rename(columns={'value' : '업종(소)'}, inplace=True)
# full.drop(['code', 'itype_s'], axis=1, inplace=True)

In [22]:
# post = pd.read_csv('/home/skinet/hojin/신우편번호_광역시도명.csv')
# post['신.우편번호 앞 2자리'] = post['신.우편번호 앞 2자리'].astype(str)
# post['신.우편번호 앞 2자리'] = post['신.우편번호 앞 2자리'].str.zfill(2)

In [23]:
# full['temp'] = full['gad_pstlz'].str[0:2]

In [24]:
# full = pd.merge(full, post, how='left', left_on='temp', right_on='신.우편번호 앞 2자리')
# full.rename(columns={'광역시도명' : '시도1'}, inplace=True)
# full.drop(['신.우편번호 앞 2자리', 'temp'], axis=1, inplace=True)

In [25]:
# cond = [
#       full['gad_add01'].str.contains('서울|특별', regex=True),
#       full['gad_add01'].str.contains('경기|경기도', regex=True),
#       full['gad_add01'].str.contains('인천', regex=True),
#       full['gad_add01'].str.contains('강원', regex=True),
#       full['gad_add01'].str.contains('충북|충청북도', regex=True),
#       full['gad_add01'].str.contains('세종', regex=True),
#       full['gad_add01'].str.contains('충남|충청남도', regex=True),
#       full['gad_add01'].str.contains('대전', regex=True),
#       full['gad_add01'].str.contains('경북|경상북도', regex=True),
#       full['gad_add01'].str.contains('대구', regex=True),
#       full['gad_add01'].str.contains('울산', regex=True),
#       full['gad_add01'].str.contains('부산', regex=True),
#       full['gad_add01'].str.contains('경남|경상남도', regex=True),
#       full['gad_add01'].str.contains('전북|전라북도', regex=True),
#       full['gad_add01'].str.contains('전남|전라남도', regex=True),
#       full['gad_add01'].str.contains('광주', regex=True),
#       full['gad_add01'].str.contains('제주', regex=True)
# ]

# choice = ['서울', '경기', '인천', '강원', '충북', '세종', '충남', '대전', 
#           '경북', '대구', '울산', '부산', '경남', '전북', '전남', '광주', '제주']

In [26]:
# full['시도2'] = np.select(cond, choice)

In [27]:
# full['시도'] = np.where(full['시도1'].isnull() == True, full['시도2'], full['시도1'])
# full.drop(['시도1', '시도2'], axis=1, inplace=True)

In [28]:
# full['monexp_amt'] = full['monexp_amt']*100

In [29]:
# bins = [0, 60000, 80000, 100000, 120000, 140000, 160000, 200000, np.inf]
# labels = ['~60k', '~80k', '~100k', '~120k', '~140k', '~160k', '~200k', '200k~']

# full['월정료'] = pd.cut(full['monexp_amt'], bins=bins, labels=labels)

In [30]:
today = datetime.today()

year = str(today.year)[2:4]
month = str(today.month - 1).zfill(2) 

first_day = today.replace(day=1)
last_day_month_ago = first_day - timedelta(days=1)


day = str(last_day_month_ago.day)

filename = year + '년' + month + '월' + '_마감.csv'

In [31]:
# full.loc[full['gad_start'] == '00000000', 'gad_start'] = '29991231'
# full.loc[full['gad_start'] == ' ', 'gad_start'] = '29991231'

In [32]:
# full['gad_start_new'] = full['gad_start'].apply(lambda x : datetime.strptime(x, '%Y%m%d'))

In [33]:
# full['유지기간_'] = full['gad_start_new'].apply(lambda x : relativedelta(last_day_month_ago, x).years)

In [34]:
# full.drop('gad_start_new', axis=1, inplace=True)

In [ ]:
# bins = [-np.inf, -0.000001, 1, 3, 5, 7, 10, np.inf]
# labels =  ['데이터 없음', '~1년', '~3년', '~5년', '~7년', '~10년', '10년~']

# full['유지기간'] = pd.cut(full['유지기간_'], bins=bins, labels=labels)

# full.drop('유지기간_', axis=1, inplace=True)

In [35]:
full.columns

Index(['kunnr_y', 'ctnum_y', 'conn_dept', 'conn_emp', 'cost_dept', 'cost_emp',
       'cstat', 'gad_name', 'gad_start', 'conn_start', 'conn_end',
       'monexp_amt', 'conn_type', 'ctype_l', 'ctype_m', 'intro_asp', 'itype_l',
       'itype_m', 'itype_s', 'itype_sdesc', 'name1', 'gad_add01', 'post_add01',
       'first_bildate', 'telf2', 'svc_str', 'pay_type', 'depo_type',
       'line_type', 'pck_str', 'b_ymd', 'sex', 'stcd2', 'gros_code',
       'gross_type', 'kvgr4', 'gad_pstlz', 'gad_add02', 'post_add02',
       'result1', 'result2', 'result3', 'result4', 'result5', 'result6',
       'sms_yn', 'email_yn', 'tm_yn', 'dm_yn', 'sk1_yn', 'sk2_yn'],
      dtype='object')

In [36]:
full = full.rename(columns={
    'conn_dept' : '경비부서',
    'ctnum_y' : '계약번호',
    'cstat' : '현상태',
    'gad_start' : '경비개시일',
    'monexp_amt' : '월정료',
    'conn_type' : '계약타입',
    'ctype_l' : '채널(대)',
    'ctype_m' : '채널(중)',
    'itype_l' : '업종(대)',
    'itype_m' : '업종(중)',
    'itype_s' : '업종(소)',
    'svc_str' : '제공서비스코드',
    'pay_type' : '납입방법',
    'depo_type' : '입금방법',
    'gross_type' : '그로스적용',
    'gros_code' : '그로스코드',
    'kvgr4' : '세그먼트',
    'gad_pstlz' : '우편번호',
    'first_bildate' : '최초청구일'
})

In [37]:
full['생년월일'] = np.nan
full['주민번호생년월일'] = np.nan
full['성별구분'] = np.nan

In [38]:
full = full[['경비부서', '계약번호', '현상태', '경비개시일', '월정료', '계약타입', '채널(대)', '채널(중)', '업종(대)', '업종(중)', '업종(소)', '제공서비스코드', 
             '납입방법', '입금방법', '생년월일', '주민번호생년월일', '성별구분', '그로스적용', '그로스코드', '세그먼트', '우편번호', '최초청구일']]

In [39]:
full.head()

,경비부서,계약번호,현상태,경비개시일,월정료,계약타입,채널(대),채널(중),업종(대),업종(중),...,납입방법,입금방법,생년월일,주민번호생년월일,성별구분,그로스적용,그로스코드,세그먼트,우편번호,최초청구일
0,551300,220065786,1,20200410,350.00,A02,A00,A01,1,10,...,2,03,NaN,NaN,NaN,,,C2,14001,20200413
1,551600,220065935,1,20200410,810.00,A01,E00,E01,4,20,...,2,0Q,NaN,NaN,NaN,,,C3,22752,20200413
2,551600,220065935,1,20200410,810.00,A01,E00,E01,4,20,...,2,0Q,NaN,NaN,NaN,,,C3,22752,20200413
3,130630,220066390,1,20200421,170.45,A01,W00,M14,4,21,...,3,0Q,NaN,NaN,NaN,,,C4,04580,20200501
4,550140,220119579,1,20200625,1000.00,A01,A00,A01,4,21,...,2,03,NaN,NaN,NaN,,,C4,15057,20200626


In [40]:
full = full.drop_duplicates()

In [41]:
full.to_csv(year+'년'+month+'월_고객현황용_마감.csv', encoding='utf-8-sig')